In [1]:
import torchvision
from torchvision import transforms

# MNIST dataset
root_path = '/home/storopoli/Downloads' # mude isso no Colab se necessário

# Pequena transformação para tensores e normalizando o tamanho
trans = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),  # força 3 canais
    transforms.ToTensor(),
    transforms.Normalize((0.1307, 0.1307, 0.1307),
                         (0.3081, 0.3081, 0.3081))])

# Train/Test Datasets
train_dataset = torchvision.datasets.MNIST(root=root_path, train=True, transform=trans, download=True)
test_dataset = torchvision.datasets.MNIST(root=root_path, train=False, transform=trans)

100%|██████████| 9.91M/9.91M [00:02<00:00, 4.78MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 130kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.23MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 8.99MB/s]


In [2]:
from torch.utils.data import DataLoader

batch_size=32

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [3]:
import torch.nn as nn
import torchvision.models as models

model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features,10)

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 217MB/s]


In [4]:
# Congela as 4 camadas
for param in model.layer1.parameters():
  param.requires_grad = False

for param in model.layer2.parameters():
  param.requires_grad = False

for param in model.layer3.parameters():
  param.requires_grad = False

for param in model.layer4.parameters(): # True para ex 3.2
  param.requires_grad = True

In [5]:
from torch.optim import Adam

# Hiperparâmetros
loss_fn = nn.CrossEntropyLoss()
learning_rate = 0.001
epochs = 1

# Instânciar o Otimizador Adam
optimizer = Adam([
    {"params": model.layer4.parameters(), "lr": 1e-5},  # fine-tuning devagar
    {"params": model.fc.parameters(), "lr": 1e-3},      # última camada mais rápido
])

In [6]:
# Isto tem que retornar True
import torch
torch.cuda.is_available()

True

In [7]:
# Treinar o Modelo
total_step = len(train_loader) # quantos batches eu tenho

# Listas vazias
loss_list = []
acc_list = []

for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Gera a propagação (feed forward)
        outputs = model(images)

        # Calcula a função-custo
        loss = loss_fn(outputs, labels)
        loss_list.append(loss.item())

        # Retro-propagação (Backprop) e a otimização com Adam
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Acurácia
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)
        if (i + 1) % 100 == 0:
            print(f"Época [{epoch+1}/{epochs}], Step [{i+1}/{total_step}], Custo: {round(loss.item(), 3)}, Acurácia: {round((correct / total) * 100, 3)}")

Época [1/1], Step [100/1875], Custo: 0.514, Acurácia: 87.5
Época [1/1], Step [200/1875], Custo: 0.081, Acurácia: 100.0
Época [1/1], Step [300/1875], Custo: 0.043, Acurácia: 100.0
Época [1/1], Step [400/1875], Custo: 0.062, Acurácia: 100.0
Época [1/1], Step [500/1875], Custo: 0.018, Acurácia: 100.0
Época [1/1], Step [600/1875], Custo: 0.179, Acurácia: 96.875
Época [1/1], Step [700/1875], Custo: 0.066, Acurácia: 96.875
Época [1/1], Step [800/1875], Custo: 0.015, Acurácia: 100.0
Época [1/1], Step [900/1875], Custo: 0.299, Acurácia: 93.75
Época [1/1], Step [1000/1875], Custo: 0.074, Acurácia: 96.875
Época [1/1], Step [1100/1875], Custo: 0.003, Acurácia: 100.0
Época [1/1], Step [1200/1875], Custo: 0.242, Acurácia: 90.625
Época [1/1], Step [1300/1875], Custo: 0.143, Acurácia: 90.625
Época [1/1], Step [1400/1875], Custo: 0.14, Acurácia: 93.75
Época [1/1], Step [1500/1875], Custo: 0.01, Acurácia: 100.0
Época [1/1], Step [1600/1875], Custo: 0.059, Acurácia: 96.875
Época [1/1], Step [1700/1875],

In [8]:
model.eval() # coloca o modelo em modo de avaliação (sem calcular gradientes)

with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        # Feed-forward com as imagens de teste
        outputs = model(images)

        # gera predições usando a função max()
        _, predicted = torch.max(outputs.data, 1)

        # Acumula total e corretas
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Acurácia do Modelo em 10k imagens de teste: {round((correct / total) * 100, 3)}")

Acurácia do Modelo em 10k imagens de teste: 98.91
